In [1]:
from pathlib import Path
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# Path to project root
PROJECT_ROOT = Path.home() / "Desktop" / "Insurance_Project"

# Add root to Python path
sys.path.append(str(PROJECT_ROOT))

In [2]:
from pathlib import Path
import os
import importlib
from src.data import preprocessing as pr
import re
file_path = PROJECT_ROOT/"data"/"raw"/"insurance_claims.csv"
data = pr.load_data(file_path)
# Feature engineering specific to the prototype dataset
data = data.drop('_c39', axis=1)
data.replace('?', np.nan, inplace=True)

for char in ['Y', 'YES']:
    data.replace(char, True, inplace=True)
for char in ['N', 'NO']:
    data.replace(char, False, inplace=True)


C:\Users\phili\AppData\Local\Temp\ipykernel_18044\438751034.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace(char, False, inplace=True)


In [3]:
null_count = int(data.isna().sum().sum()) 
print(f"Null count: {null_count}")

Null count: 972


In [4]:
missing_values = pd.DataFrame(data.isnull().sum().sort_values(ascending=False)) # type: ignore
missing_values

,0
property_damage,360
police_report_available,343
collision_type,178
authorities_contacted,91
months_as_customer,0
age,0
policy_number,0
policy_deductable,0
policy_csl,0
policy_state,0


In [5]:
target = 'total_claim_amount'
X_train, X_test, X_val, y_train, y_test, y_val = pr.split_data(data, target)# type: ignore

In [6]:
data['policy_bind_date'] = pd.to_datetime(data['policy_bind_date'], yearfirst=True)
data['month'] = data['policy_bind_date'].dt.month
data['month_sin'] = np.sin(2 * np.pi * data['month']/12)
data['month_cos'] = np.cos(2 * np.pi * data['month']/12)

data = data.drop('month', axis=1)

In [7]:

data['insured_zip'] = data['insured_zip'].astype(object)
date_cols = data.select_dtypes(include="datetime").columns# type: ignore
bool_cols = []
for col in data.columns:
    unique_vals = set(data[col].dropna().unique())
    if unique_vals.issubset({True, False}):
        data[col] = data[col].astype('boolean')  # nullable bool
        bool_cols.append(col)

for col in bool_cols:
    print(f"Bool Column --------- {col}")
print("\n----------------------------------------------------\n")

for col in date_cols:
    print(f"Date Column --------- {col}")
print("\n----------------------------------------------------\n")
num_cols = data.loc[:,~data.columns.isin(date_cols)].select_dtypes(include="number").columns# type: ignore
for col in num_cols:
    print(f"Numerical Column --------- {col}")

print("\n----------------------------------------------------\n")
cat_cols = data.select_dtypes(include="object").columns# type: ignore
cat_cols = [x for x in cat_cols if x not in bool_cols]
for col in cat_cols:
    print(f"Categorical Column --------- {col}")

num_cols.drop("policy_number")

data = data.drop('policy_bind_date', axis=1)
# Cyclical interpretation of datetime feature

Bool Column --------- property_damage
Bool Column --------- police_report_available
Bool Column --------- fraud_reported

----------------------------------------------------

Date Column --------- policy_bind_date

----------------------------------------------------

Numerical Column --------- months_as_customer
Numerical Column --------- age
Numerical Column --------- policy_number
Numerical Column --------- policy_deductable
Numerical Column --------- policy_annual_premium
Numerical Column --------- umbrella_limit
Numerical Column --------- capital-gains
Numerical Column --------- capital-loss
Numerical Column --------- incident_hour_of_the_day
Numerical Column --------- number_of_vehicles_involved
Numerical Column --------- bodily_injuries
Numerical Column --------- witnesses
Numerical Column --------- total_claim_amount
Numerical Column --------- injury_claim
Numerical Column --------- property_claim
Numerical Column --------- vehicle_claim
Numerical Column --------- auto_year
Nu

In [8]:
print(X_train)

     months_as_customer  age  policy_number policy_bind_date policy_state  \
541                 239   41         743092       2013-11-11           OH   
440                 108   31         492224       2005-12-09           IN   
482                 116   30         996253       2001-11-29           IN   
422                   8   21         355085       2012-10-09           IN   
778                 161   38         192524       2004-01-02           IL   
..                  ...  ...            ...              ...          ...   
106                 464   61         632627       1990-10-07           OH   
270                 369   55         577810       2013-04-15           OH   
860                 230   42         175960       2004-11-16           IN   
435                 102   28         810189       1999-08-29           OH   
102                 279   41         389238       2001-06-06           IL   

    policy_csl  policy_deductable  policy_annual_premium  umbrella_limit  \

In [9]:
# data = data.drop('policy_number', axis=1)
# data = data.drop('incident_city', axis=1)
# data = data.drop('incident_location', axis=1)
# data = data.drop('insured_zip', axis=1)
# data = data.drop('insured_hobbies', axis=1)
# data = data.drop('injury_claim', axis=1)
# data = data.drop('property_claim', axis=1)
# data = data.drop('vehicle_claim', axis=1)
# data = data.drop('incident_date', axis=1)




In [10]:

# numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
# ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level', 'incident_type','incident_severity',]
# ordinal_categories = [['OH', 'IN', 'IL'],
#                       ['100/300', '250/500', '500/1000'],
#                       ['MALE', 'FEMALE'],
#                       ['High School', 'Masters','JD', 'MD'],
#                       ['Single Vehicle Collision', 'Multi-vehicle Collision'],
#                       ['Minor Damage', 'Major Damage', 'Total Loss']]
# nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [11]:
data = data.drop('policy_number', axis=1)
data = data.drop('incident_city', axis=1)
data = data.drop('incident_location', axis=1)
data = data.drop('insured_zip', axis=1)
data = data.drop('insured_hobbies', axis=1)
data = data.drop('injury_claim', axis=1)
data = data.drop('property_claim', axis=1)
data = data.drop('vehicle_claim', axis=1)
data = data.drop('incident_date', axis=1)
data = data.drop('collision_type', axis=1)
data = data.drop('number_of_vehicles_involved', axis=1)
data = data.drop('incident_severity', axis=1)
data = data.drop('incident_type', axis=1)
data = data.drop('bodily_injuries', axis=1)
data = data.drop('property_damage', axis=1)
data = data.drop('police_report_available', axis=1)
data = data.drop('authorities_contacted', axis=1)
data = data.drop('incident_hour_of_the_day', axis=1)
data = data.drop('fraud_reported', axis=1)
data = data.drop('capital-gains', axis=1)
data = data.drop('capital-loss', axis=1)



In [12]:
numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level']
ordinal_categories = [['OH', 'IN', 'IL'],
                      ['100/300', '250/500', '500/1000'],
                      ['MALE', 'FEMALE'],
                      ['High School', 'Masters','JD', 'MD'],]
nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [13]:
print(numerical_columns)

Index(['months_as_customer', 'age', 'policy_number', 'policy_deductable',
       'policy_annual_premium', 'umbrella_limit', 'insured_zip',
       'capital-gains', 'capital-loss', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'bodily_injuries', 'witnesses',
       'injury_claim', 'property_claim', 'vehicle_claim', 'auto_year'],
      dtype='object')


In [14]:
importlib.reload(pr)
preprocessor, X_train_processed = pr.prepreprocessing_pipeline(
    X = X_train,
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols
)

In [15]:
importlib.reload(pr)
global_mean = y_train.mean()
X_val_processed = pr.apply_preprocessing_pipeline(X_val,
                                                  preprocessor=preprocessor)

X_test_processed = pr.apply_preprocessing_pipeline(X_test,
                                                  preprocessor=preprocessor)

In [16]:
importlib.reload(pr)
X_train_processed, y_train = pr.remove_null(X_train_processed, y_train)
X_val_processed, y_val = pr.remove_null(X_val_processed, y_val)
X_test_processed, y_test = pr.remove_null(X_test_processed, y_test)

In [17]:
train_data = pd.DataFrame(X_train_processed)
test_data = pd.DataFrame(X_test_processed)
val_data = pd.DataFrame(X_val_processed)
processed_data = pd.concat([
    train_data,
    test_data,
    val_data
], ignore_index=True)   

In [18]:
processed_data.shape

(1000, 38)

In [19]:
print(processed_data.isnull().sum().sum())

0


In [20]:
i = 0
for df in [train_data, test_data, val_data]:
    print(f"Null values in {i}: {df.isnull().sum().sum()}")
    i += 1

Null values in 0: 0
Null values in 1: 0
Null values in 2: 0


In [21]:
y_train_data = pd.Series(y_train)
y_test_data = pd.Series(y_test)
y_val_data = pd.Series(y_val)

In [22]:
y_train_data

541    10790
440    57330
482    59670
422    91260
778    60480
       ...  
106    79800
270    85300
860    58850
435    73400
102    28800
Name: total_claim_amount, Length: 700, dtype: int64

In [23]:
data_folder = PROJECT_ROOT/"data"/"processed"
train_data.to_csv(data_folder/"X_train.csv", index=False)
test_data.to_csv(data_folder/"X_test.csv", index=False)
val_data.to_csv(data_folder/"X_val.csv",index=False)
y_train_data.to_csv(data_folder/"y_train.csv",index=False)
y_test_data.to_csv(data_folder/"y_test.csv",index=False)
y_val_data.to_csv(data_folder/"y_val.csv",index=False)

In [24]:
# numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
# ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level', 'incident_type','incident_severity',]
# ordinal_categories = [['OH', 'IN', 'IL'],
#                       ['100/300', '250/500', '500/1000'],
#                       ['MALE', 'FEMALE'],
#                       ['High School', 'Masters','JD', 'MD'],
#                       ['Single Vehicle Collision', 'Multi-vehicle Collision'],
#                       ['Minor Damage', 'Major Damage', 'Total Loss']]
# nominal_columns = [x for x in cat_cols if x not in ordinal_columns]


In [25]:
numerical_columns = X_train.select_dtypes(include='number').columns# type: ignore
ordinal_columns = ['policy_state','policy_csl','insured_sex', 'insured_education_level']
ordinal_categories = [['OH', 'IN', 'IL'],
                      ['100/300', '250/500', '500/1000'],
                      ['MALE', 'FEMALE'],
                      ['High School', 'Masters','JD', 'MD'],]
nominal_columns = [x for x in cat_cols if x not in ordinal_columns]

In [26]:
numerical_columns = [col for col in numerical_columns if col in data.columns]
ordinal_columns = [col for col in ordinal_columns if col in data.columns]
nominal_columns = [col for col in nominal_columns if col in data.columns]
bool_cols = [col for col in bool_cols if col in data.columns]

In [27]:
print(numerical_columns)

['months_as_customer', 'age', 'policy_deductable', 'policy_annual_premium', 'umbrella_limit', 'witnesses', 'auto_year']


In [28]:
from sklearn.model_selection import train_test_split
target = 'total_claim_amount'
X, y = data[[col for col in data.columns if col not in target]], data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

preprocessor, X_train_processed = pr.prepreprocessing_pipeline(
    X = X_train,
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols
)
X_test_processed = pr.apply_preprocessing_pipeline(X_test, preprocessor)
train_reg, test_reg = pd.DataFrame(X_train_processed), pd.DataFrame(X_test_processed)
y_train_reg, y_test_reg = pd.Series(y_train), pd.Series(y_test)

train_reg.to_csv(data_folder/"reg"/"X_train.csv", index=False)
test_reg.to_csv(data_folder/"reg"/"X_test.csv", index=False)
y_train_reg.to_csv(data_folder/"reg"/"y_train.csv", index=False)
y_test_reg.to_csv(data_folder/"reg"/"y_test.csv", index=False)


In [29]:
print(X.shape)

(1000, 18)


In [30]:
features = X.columns
features

Index(['months_as_customer', 'age', 'policy_state', 'policy_csl',
       'policy_deductable', 'policy_annual_premium', 'umbrella_limit',
       'insured_sex', 'insured_education_level', 'insured_occupation',
       'insured_relationship', 'incident_state', 'witnesses', 'auto_make',
       'auto_model', 'auto_year', 'month_sin', 'month_cos'],
      dtype='object')

In [31]:
X_train = pd.DataFrame(X_train_processed)

In [32]:
X_test = pd.DataFrame(X_test_processed)

In [33]:
X_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.157211,-0.108809,1.408091,-0.158007,-0.488428,0.460951,-1.178365,0.0,1.0,1.0,1.0,59.0,143.0,98.0,69.0,25.0
1,2.040995,1.970113,-0.215589,-1.071160,1.677017,1.366751,0.314169,2.0,0.0,0.0,-1.0,54.0,139.0,223.0,58.0,34.0
2,0.682261,0.766527,1.408091,0.230045,-0.488428,1.366751,-0.183342,2.0,2.0,1.0,-1.0,59.0,143.0,232.0,69.0,20.0
3,1.019781,0.547693,-0.215589,-0.413392,-0.488428,1.366751,0.811680,2.0,0.0,1.0,-1.0,84.0,143.0,232.0,58.0,34.0
4,0.422631,0.766527,-1.027428,0.413773,-0.488428,-1.350648,-0.680853,2.0,0.0,1.0,3.0,59.0,143.0,232.0,58.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-1.022647,-1.421813,-1.027428,0.514830,2.543196,-0.444848,0.977517,0.0,0.0,0.0,2.0,67.0,159.0,223.0,67.0,26.0
96,0.370705,0.000608,-1.027428,-1.846037,-0.488428,1.366751,-0.349179,2.0,0.0,1.0,-1.0,51.0,139.0,223.0,71.0,36.0
97,0.794768,0.766527,-0.215589,-0.474157,2.543196,0.460951,1.309192,0.0,0.0,0.0,1.0,66.0,161.0,232.0,56.0,14.0
98,-1.438056,-1.202979,-0.215589,-0.973339,-0.488428,-0.444848,-0.183342,0.0,1.0,0.0,1.0,59.0,161.0,96.0,66.0,19.0


In [34]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,-0.832251,-0.655894,1.408091,-2.304067,-0.488428,-0.444848,0.977517,2.0,0.0,1.0,2.0,61.0,161.0,203.0,69.0,29.0
1,0.050493,-0.218226,-1.027428,0.862386,-0.488428,1.366751,-0.846690,0.0,2.0,1.0,3.0,67.0,169.0,232.0,63.0,19.0
2,0.154346,0.000608,-0.215589,-0.100027,-0.488428,-1.350648,-1.012527,2.0,1.0,0.0,0.0,61.0,169.0,223.0,58.0,34.0
3,-0.356261,-0.327643,-0.215589,1.007542,-0.488428,-0.444848,1.309192,1.0,1.0,0.0,3.0,62.0,143.0,203.0,63.0,17.0
4,-0.555312,-0.874728,-1.027428,-2.245963,2.110107,0.460951,1.640866,1.0,2.0,0.0,0.0,84.0,129.0,203.0,77.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,-0.607238,-0.765311,-1.027428,1.454762,-0.488428,-0.444848,1.640866,2.0,2.0,1.0,3.0,62.0,169.0,98.0,66.0,18.0
896,-1.161117,-1.421813,-0.215589,-0.236297,1.677017,0.460951,1.143354,0.0,2.0,0.0,-1.0,84.0,169.0,232.0,69.0,20.0
897,-0.209137,-0.327643,-1.027428,0.062696,-0.488428,-0.444848,-1.178365,2.0,0.0,0.0,-1.0,76.0,139.0,232.0,63.0,17.0
898,-1.766922,-0.765311,-0.215589,0.490917,-0.488428,-0.444848,-0.515016,0.0,2.0,0.0,-1.0,67.0,139.0,232.0,59.0,29.0


In [35]:
importlib.reload(pr)
processed_data = pr.clean_data(data=data)

In [36]:
print(processed_data.isna().sum().sum())

0


In [37]:
processed_data.to_csv(PROJECT_ROOT/"dashboard"/"assets"/"dashboard_data.csv")

In [38]:
processed_data

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_relationship,incident_state,witnesses,total_claim_amount,auto_make,auto_model,auto_year,month_sin,month_cos
0,328.0,48.0,OH,250/500,1000.0,1406.91,0.0,MALE,MD,craft-repair,husband,SC,2.0,71610.0,Saab,92x,2004.0,-8.660254e-01,5.000000e-01
1,228.0,42.0,IN,250/500,2000.0,1197.22,5000000.0,MALE,MD,machine-op-inspct,other-relative,VA,0.0,5070.0,Mercedes,E400,2007.0,1.224647e-16,-1.000000e+00
2,134.0,29.0,OH,100/300,2000.0,1413.14,5000000.0,FEMALE,PhD,sales,own-child,NY,3.0,34650.0,Dodge,RAM,2007.0,-1.000000e+00,-1.836970e-16
3,256.0,41.0,IL,250/500,2000.0,1415.74,6000000.0,FEMALE,PhD,armed-forces,unmarried,OH,2.0,63400.0,Chevrolet,Tahoe,2014.0,5.000000e-01,-8.660254e-01
4,228.0,44.0,IL,500/1000,1000.0,1583.91,6000000.0,MALE,Associate,sales,unmarried,NY,1.0,6500.0,Accura,RSX,2009.0,1.224647e-16,-1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,3.0,38.0,OH,500/1000,1000.0,1310.80,0.0,FEMALE,Masters,craft-repair,unmarried,NC,1.0,87200.0,Honda,Accord,2006.0,-5.000000e-01,-8.660254e-01
996,285.0,41.0,IL,100/300,1000.0,1436.79,0.0,FEMALE,PhD,prof-specialty,wife,SC,3.0,108480.0,Volkswagen,Passat,2015.0,5.000000e-01,8.660254e-01
997,130.0,34.0,OH,250/500,500.0,1383.49,3000000.0,FEMALE,Masters,armed-forces,other-relative,NC,3.0,67500.0,Suburu,Impreza,1996.0,8.660254e-01,5.000000e-01
998,458.0,62.0,IL,500/1000,2000.0,1356.92,5000000.0,MALE,Associate,handlers-cleaners,wife,NY,1.0,46980.0,Audi,A5,1998.0,-5.000000e-01,8.660254e-01


In [39]:
importlib.reload(pr)

<module 'src.data.preprocessing' from 'C:\\Users\\phili\\Desktop\\Insurance_Project\\src\\data\\preprocessing.py'>

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

preprocessor_pickle = pr.unfit_prepreprocessing_pipeline(
    numerical_cols=numerical_columns,
    ordinal_cols=ordinal_columns,
    ordinal_categories=ordinal_categories,
    nominal_cols=nominal_columns,
    bool_cols=bool_cols)
preprocessor_pickle.fit(X)

,transformers,"[('num', ...), ('ord', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'median'
,fill_value,None


In [44]:
import joblib
joblib.dump(preprocessor_pickle, PROJECT_ROOT/"models"/"preprocessor.pkl")
joblib.dump(features, PROJECT_ROOT/"models"/"features.pkl")


['C:\\Users\\phili\\Desktop\\Insurance_Project\\models\\features.pkl']